In [ ]:
import itertools
import json
import pathlib
import pickle

import attr
import config
import numpy as np
import pandas as pd
import sklearn.utils
import tqdm.notebook

import skrough
from skrough.metrics.gini_impurity import gini_impurity
from skrough.utils.group_index import (
    compute_dec_distribution,
    compute_homogeneity,
    split_groups,
)

In [ ]:
DATA_DIR = pathlib.Path(config.DATA_DIR)
TMP_DIR = pathlib.Path(config.TMP_DIR)
SEP = ";"

FILEPATH_IN = DATA_DIR / "train_utf_with_reordered_cols.csv"

In [ ]:
def __prepare_values(values):
    factorized_values, uniques = pd.factorize(values)
    uniques = len(uniques)
    return factorized_values, uniques


def prepare_df(filepath=None, df=None):
    if df is None:
        df = pd.read_csv(filepath, sep=SEP)
    df_dec = df.pop("target")
    df = df.astype("category")
    df = df.apply(lambda x: x.cat.codes)

    x = df
    y = df_dec
    x, y = sklearn.utils.check_X_y(x, y, multi_output=False)
    data = np.apply_along_axis(__prepare_values, 0, x)
    x = np.vstack(data[0]).T
    x_count_distinct = data[1]
    y, y_count_distinct = __prepare_values(y)

    return x, x_count_distinct, y, y_count_distinct


def _compute_chaos_score(group_index, n_groups, xx, yy, yy_count_distinct):
    distribution = compute_dec_distribution(
        group_index, n_groups, yy, yy_count_distinct
    )
    return gini_impurity(distribution, len(xx))


def get_chaos_score(xx, xx_count_distinct, yy, yy_count_distinct, attrs):
    group_index = np.zeros(len(xx), dtype=np.int_)
    n_groups = 1
    for attr in attrs:
        group_index, n_groups = split_groups(
            group_index,
            n_groups,
            xx[:, attr],
            xx_count_distinct[attr],
            compress_group_index=True,
        )
    result = _compute_chaos_score(group_index, n_groups, xx, yy, yy_count_distinct)
    return result


def get_results(data_filepath, bireducts_filepath, local_scope=False, sep=SEP):
    df_columns = pd.read_csv(data_filepath, sep=sep, nrows=0).columns[:-1]

    xx, xx_count_distinct, yy, yy_count_distinct = prepare_df(data_filepath)
    results = {}
    filepath = bireducts_filepath
    with filepath.open("r") as f:
        bireducts = json.load(f)
    counts = np.zeros(xx.shape[1])
    scores = np.zeros(xx.shape[1])
    scores_2 = np.zeros(xx.shape[1])
    for bireduct in tqdm.notebook.tqdm(bireducts):
        bireduct_objects = bireduct["objects"]
        bireduct_all_attrs = set(bireduct["attributes"])
        xxx = xx
        yyy = yy
        if local_scope:
            xxx = xxx[bireduct_objects]
            yyy = yyy[bireduct_objects]
        starting_chaos_score = get_chaos_score(
            xxx, xx_count_distinct, yyy, yy_count_distinct, bireduct_all_attrs
        )
        counts[bireduct["attributes"]] += 1
        for attr in bireduct["attributes"]:
            attrs_to_check = bireduct_all_attrs.difference([attr])
            current_chaos_score = get_chaos_score(
                xxx, xx_count_distinct, yyy, yy_count_distinct, attrs_to_check
            )
            score_val = current_chaos_score - starting_chaos_score
            scores[attr] += score_val
            scores_2[attr] += score_val * len(bireduct_objects) / xx.shape[0]
    avg = np.divide(scores, counts, out=np.zeros_like(scores), where=counts > 0)
    avg_2 = np.divide(scores_2, counts, out=np.zeros_like(scores_2), where=counts > 0)
    results[filepath.name] = pd.DataFrame(
        {
            "column": df_columns,
            "score": scores,
            "score2": scores_2,
            "count": counts,
            "avg": avg,
            "avg2": avg_2,
        }
    ).sort_values(["score"], ascending=False)
    del bireducts
    return results


def do_eval(data_filepath, bireducts_filepath, scope):
    data_filepath = pathlib.Path(data_filepath)
    bireducts_filepath = pathlib.Path(bireducts_filepath)

    assert scope in ("global", "local")
    results_dir = bireducts_filepath.parent
    results_filename = f"{bireducts_filepath.stem}_eval_scope_{scope}.pkl"
    results = get_results(
        data_filepath,
        bireducts_filepath,
        local_scope=True if scope == "local" else False,
    )
    with (results_dir / results_filename).open("wb") as f:
        pickle.dump(results, f)

In [ ]:
# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_100_allowed_randomness_0.1_max_n_attrs_3.json',
#         'local')
# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_100_allowed_randomness_0.1_max_n_attrs_3.json',
#         'global')

# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_100_allowed_randomness_0.1_max_n_attrs_None.json',
#         'local')
# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_100_allowed_randomness_0.1_max_n_attrs_None.json',
#         'global')

# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_30_allowed_randomness_0.2_max_n_attrs_3.json',
#         'local')
# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_30_allowed_randomness_0.2_max_n_attrs_3.json',
#         'global')

# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_30_allowed_randomness_0.2_max_n_attrs_None.json',
#         'local')
# do_eval(DATA_DIR / 'train_utf_with_reordered_cols.csv',
#         TMP_DIR / 'compare/bireducts_daab_reordered_cols_n_1000_candidate_attrs_30_allowed_randomness_0.2_max_n_attrs_None.json',
#         'global')